In [1]:
import json
import datetime
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def read_output(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)

    results = pd.DataFrame(data.get("Payload")['body'])
    cols = ["execution_time", "cost", "glue_job_name", "use_case", "proportion", "scale"]
    results = results[cols]
    results.rename({"glue_job_name": "compute"}, axis=1, inplace=True)
    results["execution_time_min"] = results["execution_time"] / 60
    results = results.groupby(["compute", "use_case", "proportion"])[["execution_time_min", "cost"]].mean().reset_index()
    return results

# Hudi vs Iceberg

## Maintainability

In trying to understand maintainability we have consideorange a few things:

- **complexity:**
  - less complex code is easier to pickup, understand and maintain.
- **community uptake:**
  - greater community uptake indicates longevity of a tool and wider community support (more answers on stack-overflow!)
- **documentation:**
  - well documented tools are easier to understand and use

### Code Complexity

Measuring code complexity can be a little subjective however there are tools around which allow you to calcuate empirical measure from code:

- Logical Lines Of Code (LLOC): a better measure of volume of code than simply counting lines
- Cyclomatic Complexity (CC): a score corresponding to the number of "decisions" a block of code contains
- Maintainability Index (MI): a score computed from several measure (including LLOC and CC), which is meant to indicate how easy to support and change the source code is.

In [3]:
%%writefile example.py
x = 100; if x > 50: print("foobar"); else: print("barfoo")

Writing example.py


In [4]:
!radon raw example.py

example.py
    LOC: 1
    LLOC: 5
    SLOC: 1
    Comments: 0
    Single comments: 0
    Multi: 0
    Blank: 0
    - Comment Stats
        (C % L): 0%
        (C % S): 0%
        (C + M % L): 0%


In [5]:
%%writefile example.py
def some_function(a, b):
    if a > b:
        return "foobar"
    else:
        return "barfoo"

Overwriting example.py


In [6]:
!radon cc example.py -s

example.py
    F 1:0 some_function - A (2)


In [7]:
!radon mi example.py -s

example.py - A (79.74)


In [8]:
code_complexity = pd.DataFrame([
    {"compute": "existing_glue",  "measure": "LLOC",      "value": 379},
    {"compute": "existing_glue",  "measure": "num_funcs", "value": 31},
    {"compute": "existing_glue",  "measure": "CC",        "value": 3.16},
    {"compute": "existing_glue",  "measure": "MI",        "value": 41.36},

    {"compute": "athena_iceberg",  "measure": "LLOC",      "value": 99},
    {"compute": "athena_iceberg",  "measure": "num_funcs", "value": 7},
    {"compute": "athena_iceberg",  "measure": "CC",        "value": 1.71},
    {"compute": "athena_iceberg",  "measure": "MI",        "value": 50.29},

    {"compute": "glue_iceberg",  "measure": "LLOC",      "value": 91},
    {"compute": "glue_iceberg",  "measure": "num_funcs", "value": 4},
    {"compute": "glue_iceberg",  "measure": "CC",        "value": 1.25},
    {"compute": "glue_iceberg",  "measure": "MI",        "value": 64.21},

    {"compute": "glue_hudi",  "measure": "LLOC",      "value": 99},
    {"compute": "glue_hudi",  "measure": "num_funcs", "value": 5},
    {"compute": "glue_hudi",  "measure": "CC",        "value": 3.2},
    {"compute": "glue_hudi",  "measure": "MI",        "value": 63.26},
])

px.bar(code_complexity, x="measure", y="value", color="compute", barmode="group")

- from the plot above theres little difference between the options, based on empiriacal measures.
- much simpler than the existing job (though not necessarily a fair comparison).
- athena solution is in SQL rather than in python/spark - different rather than better/worse.
- glue/spark has more scope for fine-tuning/optimisation than athena, which we haven't implemented - this could add significant complexity.
- however glue was designed for ETL so if using athena there might be additional things to consider.
- with athena we also don't need to deal with server configurations (num-workers etc.)

### Community Uptake

#### google trends

![image](pics/google_trends.png)

#### github stars
[![Star History Chart](https://api.star-history.com/svg?repos=apache/iceberg,apache/hudi&type=Date)](https://star-history.com/#apache/iceberg&apache/hudi&Date)

#### github pulse - hudi

![image](pics/hudi_github_pulse.png)

#### github pulse - iceberg

![image](pics/iceberg_github_pulse.png)

### Documentation

[iceberg](https://iceberg.apache.org/docs/1.2.0/)

[hudi](https://hudi.apache.org/docs/overview/)

Overall hudi and iceberg both have good documention.

However, we have found both with glue/spark and athena/iceberg that there are some optimisations and configurations which can make a big impact but are not documented.

## Results

We made the decision last week to drop our investigation into hudi and focus on iceberg comparing its implementation with glue vs athena. This was driven by the conclusion that icebergs better integration within AWS and athena outweighed any other advantages hudi provides (i.e. better support for streaming data).

### scale = 100 (~290million rows)

- attempted scd2 on update proportions of [0.001, 0.01, 0.1, 0.99]
- glue+iceberg solution was able to handle all of these update sized
- athena+iceberg solution handled all but the 0.99 update size


![image](pics/scale_100_state_machine.png)

### scale = 3000 (~8billion rows)

- attempted scd2 on update proportions of [0.001, 0.01, 0.1, 0.99]
- glue+iceberg solution was unable to handle any of these update sizes
- athena+iceberg solution handled updates on 0.001 and 0.01 proportions


![image](pics/scale_3000_state_machine.png)

Within our production data the largest tables are ~3billion rows, well below the 8-billion scale we tested against and the number of cdc's are usually >1% of all rows.

### Bulk Insert Comparison

In [3]:
scale_100 = read_output(output_file="output/output_100_scale.json")
scale_3000 = read_output(output_file="output/output_3000_scale.json")

df_athena_100 = scale_100.loc[(scale_100["use_case"]=="bulk_insert") & (scale_100["compute"]=="athena_iceberg")]
df_pyspark_100 = scale_100.loc[(scale_100["use_case"]=="bulk_insert") & (scale_100["compute"]=="glue_iceberg")]
df_athena_3000 = scale_3000.loc[(scale_3000["use_case"]=="bulk_insert") & (scale_3000["compute"]=="athena_iceberg")]
df_pyspark_3000 = scale_3000.loc[(scale_3000["use_case"]=="bulk_insert") & (scale_3000["compute"]=="glue_iceberg")]

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=["Execution Time (mins)", "Cost ($)"],
                    horizontal_spacing = 0.05)
fig.add_trace(go.Scatter(x=df_athena_100["proportion"], 
                         y=df_athena_100["execution_time_min"], 
                         name="athena - 0.1TB", 
                         marker={"color": "blue", "size":10}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_athena_100["proportion"], 
                         y=df_athena_100["cost"], 
                         marker={"color": "blue", "size":10},
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=df_pyspark_100["proportion"], 
                         y=df_pyspark_100["execution_time_min"],
                         name="pyspark - 0.1TB", 
                         marker={"color": "orange", "size":10}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_pyspark_100["proportion"], 
                         y=df_pyspark_100["cost"],
                         marker={"color": "orange", "size":10},
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=df_athena_3000["proportion"], 
                         y=df_athena_3000["execution_time_min"], 
                         name="athena - 3TB", 
                         marker={"color": "blue", "symbol":"square", "size":10},
                         line={'dash':'dash'}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_athena_3000["proportion"], 
                         y=df_athena_3000["cost"], 
                         marker={"color": "blue", "symbol":"square", "size":10},
                         showlegend=False,
                         line={'dash':'dash'}),
              row=1,col=2)
fig.add_trace(go.Scatter(x=df_pyspark_3000["proportion"], 
                         y=df_pyspark_3000["execution_time_min"],
                         name="pyspark - 3TB", 
                         marker={"color": "orange", "size":10, "symbol":"square",},
                         line={'dash':'dash'}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_pyspark_3000["proportion"], 
                         y=df_pyspark_3000["cost"],
                         marker={"color": "orange", "size":10, "symbol":"square",},
                         showlegend=False,
                         line={'dash':'dash'}),
              row=1,col=2)
fig.update_xaxes(showticklabels=False)
# fig.update_xaxes(title_text="Attempt")
fig.update_layout(margin=dict(l=0, r=0, t=20, b=1))


### SCD2 Comparison

In [6]:
scale_100 = read_output(output_file="output/output_100_scale.json")
scale_3000 = read_output(output_file="output/output_3000_scale.json")

df_athena_100 = scale_100.loc[(scale_100["use_case"]=="scd2_complex") & (scale_100["compute"]=="athena_iceberg")]
df_pyspark_100 = scale_100.loc[(scale_100["use_case"]=="scd2_complex") & (scale_100["compute"]=="glue_iceberg")]
df_athena_3000 = scale_3000.loc[(scale_3000["use_case"]=="scd2_complex") & (scale_3000["compute"]=="athena_iceberg")]

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=["Execution Time (mins)", "Cost ($)"],
                    horizontal_spacing = 0.05)
fig.add_trace(go.Scatter(x=df_athena_100["proportion"], 
                         y=df_athena_100["execution_time_min"], 
                         name="athena - 0.1TB", 
                         marker={"color": "blue", "size":10}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_athena_100["proportion"], 
                         y=df_athena_100["cost"], 
                         marker={"color": "blue", "size":10},
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=df_pyspark_100["proportion"], 
                         y=df_pyspark_100["execution_time_min"],
                         name="pyspark - 0.1TB", 
                         marker={"color": "orange", "size":10}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_pyspark_100["proportion"], 
                         y=df_pyspark_100["cost"],
                         marker={"color": "orange", "size":10},
                         showlegend=False),
              row=1,col=2)
fig.add_trace(go.Scatter(x=df_athena_3000["proportion"], 
                         y=df_athena_3000["execution_time_min"], 
                         name="athena - 3TB", 
                         marker={"color": "blue", "symbol":"square", "size":10},
                         line={'dash':'dash'}),
              row=1, col=1)
fig.add_trace(go.Scatter(x=df_athena_3000["proportion"], 
                         y=df_athena_3000["cost"],
                         marker={"color": "blue", "symbol":"square", "size":10},
                         showlegend=False,
                         line={'dash':'dash'}),
              row=1,col=2)
fig.update_xaxes(title_text="Proportion of rows updated")
fig.update_layout(margin=dict(l=0, r=0, t=20, b=1))


## Conclusions

- Investigation into impact on analytical users show that having iceberg rather than hive tables should not impact query-time/data-scanned with appropriate optimisation.
- We've demonstrated that athena+iceberg can handle CDC volumes of 80 million rows against a table of 8 billion rows
- Our current largest table is 3 billion rows with daily cdcs of ~1 million, giving us plenty of headroom with continued growth in the data
- We could build an athena based solution which could handle larger cdcs (for example when a full-refresh is requiorange) by breaking things into manageble chunks
- All done with 0 optimisation, there are likely optimisations we could make to further handle larger volumes
- Glue+Iceberg, without optimisation doesn't work at the larger volumes we've been testing against, and is drastically more expensive than using athena at lower (but still significant volumes) even though the queries are practially identical to the athena-queries. There are optimisations we could do, we'll consider doing further work on this after prioritising/planning (tomorrow).

- **Our recommendation based on this investigation is therefore that athena+iceberg is the best solution for our use cases**